In [3]:
# success
# bad resolution
import os
import cv2
import numpy as np
from tqdm import tqdm

In [4]:
# image size = 4000*2252
# resize to 4000*2250 in load_image()
piece_scale = 50 # need to be the factor of 250

original_width = 4000
original_height = 2250

image_folder = './japan/'
target_image_dir = './target/20230824_115144.jpg'

In [5]:
def load_images(folder_path, scale=1.0):
    resize_height = int(original_height / scale)
    resize_width = int(original_width / scale)
    
    resize_image_list = []
    bgr_mean_list = []

    files = os.listdir(folder_path)  
    for i in tqdm(files):     
        image = cv2.imread(folder_path+i)  
        # rotate the image 
        if image.shape[0]  > image.shape[1]:
            image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        # resize the image
        image = cv2.resize(image[1:2251, 0:4000], (resize_width, resize_height), interpolation=cv2.INTER_AREA)

        resize_image_list.append(image)
        # bgr_mean_list.append(np.mean(image, axis=(1,0)))

    return resize_image_list
# return resize images

In [6]:
resize_image_list = load_images(image_folder, piece_scale)


100%|██████████| 5213/5213 [08:04<00:00, 10.76it/s]


In [7]:
target_image = cv2.imread(target_image_dir)
print(target_image.shape)
target_image = target_image[1:2251, 0:4000] # delete this if you don't need it

(2252, 4000, 3)


In [8]:
np.savez('images_list_nome_80_45.npz', *resize_image_list)
loaded_data = np.load('images_list_nome_80_45.npz')
loaded_images = [loaded_data[key] for key in loaded_data]

# print(loaded_images[0].shape)
# print(len(loaded_images))
print(image.shape)

In [9]:
resize_height = int(original_height / piece_scale)
resize_width = int(original_width / piece_scale)

bgr_mean_list = []
for i in tqdm(loaded_images):
    bgr_mean_list.append(np.mean(i, axis=(1,0)))
print(len(bgr_mean_list))

replace = np.zeros((original_height, original_width, 3), np.uint8)

for y in tqdm(range(0, original_height, resize_height)):
    for x in range(0, original_width, resize_width):
        part = target_image[y:y+resize_height, x:x+resize_width]
        bgr = np.mean(part, axis=(1,0))
        min_distance = 195075
        for i in range(len(loaded_images)):
            j = loaded_images[i]
            k = bgr_mean_list[i]
            distance = ((bgr[0]-k[0])**2+
                        (bgr[1]-k[1])**2+
                        (bgr[2]-k[2])**2)
            if distance<min_distance:
                min_distance = distance
                choice = j  
        replace[y:y+resize_height, x:x+resize_width] = choice

# 変換後画像の保存
cv2.imwrite('replace_5.jpg', replace)  

100%|██████████| 5213/5213 [00:00<00:00, 22275.69it/s]


5213


100%|██████████| 50/50 [00:10<00:00,  4.69it/s]


True

In [12]:
from skimage.metrics import peak_signal_noise_ratio,  structural_similarity
original = cv2.imread(target_image_dir)
reconstructed = cv2.imread("replace_5.jpg")
print("PSNR", peak_signal_noise_ratio(original[1:2251, 0:4000], reconstructed))
print(original[1:2251, 0:4000].shape)
print(reconstructed.shape)
print(structural_similarity(cv2.cvtColor(original[1:2251, 0:4000], cv2.COLOR_BGR2GRAY), cv2.cvtColor(reconstructed, cv2.COLOR_BGR2GRAY)))

# 4
# PSNR 11.20746530741257
# (2250, 4000, 3)
# (2250, 4000, 3)
# 0.07700302151526704


PSNR 11.210467534576583
(2250, 4000, 3)
(2250, 4000, 3)
0.077014645198093
